In [ ]:
import mnist
from skimage.transform import resize
import numpy as np
import json
import subprocess
import tempfile
import importlib 
import os
import experiment

In [ ]:
(X_train, y_train) = (mnist.train_images(), mnist.train_labels())
(X_test, y_test) = (mnist.test_images(), mnist.test_labels())

def one_hot(v):
    z = np.zeros(10)
    z[v] = 1
    return z
y = np.array([one_hot(x) for x in np.concatenate((y_train, y_test))])
X = np.concatenate((X_train, X_test))
X = np.array([x[2:26][2:26] for x in X])
X = np.array([resize(x, (10, 10), anti_aliasing=True, mode='constant') for x in X])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(X[0])

In [ ]:
X = X.reshape(-1, 100)
X = X.astype(np.float64)
y = y.astype(np.float64)

In [ ]:
X.shape

In [ ]:
# Compile and run Futhark MNIST network
nand_dsl = 'Seq (Net 100 100) (Net 100 10)'
a, w = experiment.compile_ann(nand_dsl).main(X, y)

In [ ]:
a

In [ ]:
np.save('parameters_mnist_seq', [x.get() for x in w.data])

In [ ]:
# Compile and run parallel Futhark MNIST network
nand_dsl = 'Seq (Seq (Net 100 20) (Par (Net 20 10) (Net 20 10))) (Net 20 10))'
a, w = experiment.compile_ann(nand_dsl).main(X, y)

In [ ]:
a